In [ ]:
from flask import Flask,render_template,request
import csv
import socket
from datetime import datetime,date

nam = 'Mr P. Verma'
ID = 'faculty123'
pwd = '12345'
perm = False
data=[]
final=[]

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
s.connect(("8.8.8.8", 80))
ip = (s.getsockname()[0])
s.close()

app = Flask(__name__)

@app.route('/',methods=['POST','GET'])
def home():
    return render_template('homepage.html')

@app.route('/admin')
def admin():
    if(request.args.get('id')==ID and request.args.get('pass')==pwd):
        i=0
        global data
        data=[]
        with open('Name_List.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                i+=1
                if(i==1): continue
                data.append(row)
        return render_template('dashboard.html',name=nam,status='INACTIVE',dat = data, col = '#dd7f1e',ip=ip)
    elif(request.args.get('id')==None and request.args.get('pass')==None):
        return render_template('login.html')
    else:
        return render_template('submit.html',a='Oops !!!',b='Invalid credentials have been entered.')
    
@app.route('/activate')
def activate():
    global perm
    perm = True
    return render_template('dashboard.html',name=nam,status='ACTIVE',dat = data, col = '#48695c',ip=ip)

@app.route('/revoke')
def revoke():
    global perm
    perm = False
    return render_template('dashboard.html',name=nam,status='INACTIVE',dat = data, col = '#dd7f1e',ip=ip)

@app.route('/generate')
def generate():
    global perm
    perm = False
    i=0
    nums=dict()
    with open('temp.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            i+=1
            if(i==1): continue
            nums[row[0]]=row[2].split(' ')[1]
    global final
    final=[]
    for j in data:
        if(j[1] in nums.keys()):
            final.append(['#48695c',j[0],j[1],j[2],nums[j[1]],'Present'])
        else:
            final.append(['#dd7f1e',j[0],j[1],j[2],'NA','Absent'])
    dat = str(date.today())
    with open('Attendance_'+dat+'.csv', 'w', newline='\n') as csvfile:
        wrt = csv.writer(csvfile, delimiter=',')
        wrt.writerow(['Serial No.','Registration No.','Name','Timestamp','Status'])
        for i in final:
            wrt.writerow(i[1:])
    with open('temp.csv', 'w', newline='\n') as csvfile:
        wrt = csv.writer(csvfile, delimiter=',')
        wrt.writerow(['Registration Number','Fingerprint','Timestamp'])
    return render_template('report.html',name=nam,data=final,index='0')

@app.route('/all')
def allfunc():
    return render_template('report.html',name=nam,data=final,index='0')

@app.route('/present')
def present():
    pres=[]
    for i in final:
        if(i[5]=='Present'):
            pres.append(i)
    if(len(pres)==0):
        pres.append(['#dd7f1e','NA','NA','NA','NA','NA'])
    return render_template('report.html',name=nam,data=pres,index='1')

@app.route('/absent')
def absent():
    absent=[]
    for i in final:
        if(i[5]=='Absent'):
            absent.append(i)
    if(len(absent)==0):
        absent.append(['#dd7f1e','NA','NA','NA','NA','NA'])
    return render_template('report.html',name=nam,data=absent,index='2')

@app.route('/toggle/<id>',methods=['GET'])
def toggle(id):
    global final
    for i in range(len(final)):
        if(final[i][1]==id):
            if(final[i][5]=='Absent'): 
                final[i][5]='Present'
                now = datetime.now()
                final[i][4]= now.strftime("%H:%M:%S")
                final[i][0]='#48695c'
            else: 
                final[i][5]='Absent'
                final[i][4]='NA'
                final[i][0]='#dd7f1e'
    dat = str(date.today())
    with open('Attendance_'+dat+'.csv', 'w', newline='\n') as csvfile:
        wrt = csv.writer(csvfile, delimiter=',')
        wrt.writerow(['Serial No.','Registration No.','Name','Timestamp','Status'])
        for i in final:
            wrt.writerow(i[1:])
    return render_template('report.html',name=nam,data=final,index='0')

@app.route('/student')
def student():
    return render_template('attendance.html')

@app.route('/add', methods=['POST','GET'])
def add():
    if(perm==False):
        return '0'
    num = request.form['number']
    fp = request.form['fp']
    i=0
    numbers = []
    with open('Name_List.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            i+=1
            if(i==1): continue
            numbers.append(row[1])
    if(num not in numbers):
        return '1'
    else:
        i=0
        nums = []
        fps = []
        with open('temp.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                i+=1
                if(i==1): continue
                nums.append(row[0])
                fps.append(row[1])
        if(num in nums):
            return '2'
        elif(fp in fps):
            return '3'
        else: 
            now = datetime.now()
            time = now.strftime("%d/%m/%Y %H:%M:%S")
            with open('temp.csv', 'a', newline='\n') as csvfile:
                wrt = csv.writer(csvfile, delimiter=',')
                wrt.writerow([num] + [fp] + [time])
            return '4'

@app.route('/sub/<ans>', methods=['POST','GET'])
def sub(ans):
    if(ans=='0'):
        return render_template('submit.html',a='Oops !!!', b='Attendance link is inactive.')
    elif(ans=='1'):
        return render_template('submit.html',a='Oops !!!', b='Invalid Registration Number entered.')
    elif(ans=='2'):
        return render_template('submit.html',a='Oops !!!', b='Duplicate attendence.')
    elif(ans=='3'):
        return render_template('submit.html',a='Oops !!!', b='Device has already been used in this session.')
    else:
        return render_template('submit.html',a='Thank You', b='Your attendence has been successfully registered.')

app.run('0.0.0.0', 10001)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:10001/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Nov/2020 17:52:39] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 17:52:40] "GET /admin HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 17:52:44] "GET /admin?id=faculty123&pass=12345 HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 17:52:46] "GET /activate HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 17:52:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 17:53:42] "GET /student HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 17:53:49] "POST /add HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2020 17:53:49] "GET /sub/4 HTTP/1.1" 200 -
192.168.0.103 - - [02/Nov/2020 17:53:55] "GET / HTTP/1.1" 200 -
192.168.0.103 - - [02/Nov/2020 17:53:56] "GET /student HTTP/1.1" 200 -
192.168.0.103 - - [02/Nov/2020 17:54:02] "POST /add HTTP/1.1" 200 -
192.168.0.103 - - [02/Nov/2020 17:54:02] "GET /sub/2 HTTP/1.1" 200 -
192.168.0.103 - - [02/Nov/2020 17:54:05] "POST /add HTTP/1.1" 200 -
192.168.0.103 - - [02/Nov/2020 17:54:05] "GET /sub/4 HTTP/1.1"